In [1]:
import pandas as pd
import numpy as np

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
table_name = 'CUSTOMER_DATA'

sf_df = my_session.sql("select * from {}".format(table_name))
df_customers = sf_df.to_pandas()